<a href="https://colab.research.google.com/github/AmandinChyba/Project1-Fields-2022/blob/main/Walk_on_Boundary_and_BIEM/adamstepWOB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import jax
from jax import lax
from jax import random

import jax.numpy as jnp
import numpy as np

import time

import matplotlib.pyplot as plt
import pandas as pd

In [2]:
'''
signed distance function of the domain
'''
@jax.jit
def signDistance(p):
  return jnp.linalg.norm(p,ord=2,axis=0) - 1 # unit sphere

In [3]:
'''
bounding box diameter
'''
@jax.jit
def box():
  return 2

In [4]:
'''
Dirchlet boundary condition
'''
@jax.jit
def boundaryCond(p):
  return 1.0*(p[1] > 0.0)

In [5]:
'''
the true solution
'''
def trueSoln(p):
  return 0.5 + np.arctan(2*p[1]/(1-np.power(p[0],2)-np.power(p[1],2)))/np.pi

In [6]:
'''
binary search to find boundary intersection
'''
@jax.jit
def rootSearch(p0, p1, rootSteps, dir):
  p1 = p0 + p1 # need to center p1 at p0

  t = (jnp.zeros((jnp.size(p0, 1), jnp.size(p0, 2))), 
       jnp.ones((jnp.size(p0, 1), jnp.size(p0, 2))))
  v1 = signDistance(p0 + dir * t[1] * (p1 - p0))
  next = (t[0] * v1 + t[1] * v1) / (2 * v1)
  dist = signDistance(p0 + dir * next * (p1 - p0))
  t = (jnp.where(dist > 0, t[0], next), 
       jnp.where(dist < 0, t[1], next))

  def adamStep(i, t):
    v0 = signDistance(p0 + dir * t[0] * (p1 - p0))
    v1 = signDistance(p0 + dir * t[1] * (p1 - p0))
    next = (t[0] * v1 - t[1] * v0) / (v1 - v0)
    dist = signDistance(p0 + dir * next * (p1 - p0))
    old_t0 = t[0]
    old_t1 = t[1]
    new_t0 = jnp.where(dist > 0, t[0], next)
    new_t0 = jnp.where(jnp.isnan(new_t0), old_t0, new_t0)
    new_t1 = jnp.where(dist < 0, t[1], next)
    new_t1 = jnp.where(jnp.isnan(new_t1), old_t1, new_t1)
    new_t = (new_t0, new_t1)
    return new_t

  t = lax.fori_loop(1, rootSteps, adamStep, t)
  
  return p0 + dir * t[0] * (p1 - p0), t[0] # return the roots, t-values

In [11]:
'''
run the wob and calculate the estimator
'''
@jax.jit
def wob(p0, p1, t, rootSteps):
  '''
  pFirst, tFirst = binaryRootSearch(p0, p1[0], rootSteps, 1)
  def wobStep(i, next):
    est = next[1] + 2 * jnp.power(-1,i+1) * boundaryCond(next[0])

    pBack, tBack = binaryRootSearch(next[0], p1[i], rootSteps, -1)
    pFor, tFor = binaryRootSearch(next[0], p1[i], rootSteps, 1)
    pNext = jnp.where(tBack > tFor, pBack, pFor)
    return (pNext, est)

  est = jnp.zeros((jnp.size(p1,2),jnp.size(p1, 3)))
  pLast, est = lax.fori_loop(1, t, wobStep, (pFirst, est))
  return est + jnp.power(-1, t+1) * boundaryCond(pLast)
  '''
  runningEst = jnp.zeros((jnp.size(p0,1),jnp.size(p0,2)))
  runningSign = 1

  pNext, tFirst = rootSearch(p0, p1[0], rootSteps, 1)

  for p in p1[1:]:
    runningEst += runningSign * boundaryCond(pNext)
    runningSign *= -1
    pBack, tBack = rootSearch(pNext, p, rootSteps, -1)
    pFor, tFor = rootSearch(pNext, p, rootSteps, 1)
    pNext = jnp.where(tBack - tFor > 0, pBack, pFor)

  runningEst *= 2
  runningEst += runningSign * boundaryCond(pNext)
  return runningEst

In [8]:
'''
setup the wob
'''
def computeSoln(t, rootSteps, batches, p, key):
  p = p.transpose()
  p = jnp.reshape(p, (jnp.size(p,0), jnp.size(p,1), 1))
  p0 = jnp.tile(p, (1,1,batches))

  # generate random directions in n-dimensions
  p1 = random.normal(key, shape=(t, jnp.size(p, 0), jnp.size(p, 1), batches))
  normalize = jnp.linalg.norm(p1, ord=2, axis=1, keepdims=True)
  p1 = box() * jnp.divide(p1, normalize)

  ans = wob(p0, p1, t, rootSteps)
  return jnp.mean(ans, 1, keepdims=True)

In [28]:
%%time
'''
testing code
'''
key = random.PRNGKey(int(time.time()))
x = jnp.array([[0.6,-0.2]])

y = computeSoln(10, 15, 100000, x, key)
print('solution:', y)
print('exact:', trueSoln(x[0]))

solution: [[0.34412]]
exact: 0.3128329522748145
CPU times: user 35.4 ms, sys: 3.14 ms, total: 38.5 ms
Wall time: 35.5 ms
